In [2]:
import sys
print(sys.path)

sys.path.append('/hy-tmp/yolo_soybean/ultralytics')

['/hy-tmp/yolo_soybean/ultralytics', '/usr/local/miniconda3/lib/python38.zip', '/usr/local/miniconda3/lib/python3.8', '/usr/local/miniconda3/lib/python3.8/lib-dynload', '', '/usr/local/miniconda3/lib/python3.8/site-packages', '/hy-tmp/yolo_soybean/ultralytics']


In [3]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8m.yaml")  # build a new model from YAML
model = YOLO("/hy-tmp/yolo_soybean/ckpts/yolov8m.pt")  # load a pretrained model (recommended for training)

# Train

In [1]:
# Train the model
results = model.train(data="/hy-tmp/yolo_soybean/datasets/Normal_dataset/normal.yaml", epochs=300, imgsz=640, batch=16)

# Evaluation and inference

### Predict seeds

In [7]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8m.yaml")  # build a new model from YAML
model = YOLO("best.pt")  # load a pretrained model (recommended for training)


In [11]:
def load_labels(label_path):
    """Load label file and count the number of beans based on class ID."""
    count_map = {0: 1, 1: 2, 2: 3, 3: 4}  
    total_count = 0
    with open(label_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            class_id = int(line.split()[0])
            total_count += count_map[class_id]
    return total_count


def count_beans(predictions):
    """Count the number of beans in predictions based on class ID."""
    count_map = {0: 1, 1: 2, 2: 3, 3: 4}  
    total_count = 0
    for prediction in predictions:
        class_id = int(prediction[5])  # column 6:ID
        total_count += count_map[class_id]
    return total_count

import glob
import os
import numpy as np

test_images_path = '/hy-tmp/yolo_soybean/datasets/Normal_dataset/eval/images'
labels_path = '/hy-tmp/yolo_soybean/datasets/Normal_dataset/eval/labels'

image_paths = glob.glob(os.path.join(test_images_path, '*.*'))

errors = []
results = []

for image_path in image_paths:
    image_name = os.path.basename(image_path)
    label_name = os.path.splitext(image_name)[0] + '.txt'
    label_path = os.path.join(labels_path, label_name)

    if not os.path.exists(label_path):
        print(f'Label file not found for image {image_name}, skipping...')
        continue

    true_count = load_labels(label_path)
    model_results = model(image_path)

    if model_results and len(model_results) > 0:
        predictions = model_results[0].boxes.data  
        pred_count = count_beans(predictions.cpu().numpy())  
    else:
        pred_count = 0

    error = abs(pred_count - true_count)
    errors.append(error)
    results.append((image_name, true_count, pred_count, error))

# Sort results by error in descending order
results.sort(key=lambda x: x[3], reverse=True)

# Write results to a file
with open('sorted_errors.txt', 'w') as file:
    for result in results:
        file.write(f'Image: {result[0]}, True Count: {result[1]}, Predicted Count: {result[2]}, Error: {result[3]}\n')

mae = np.mean(errors)
print(f'Mean Absolute Error (MAE): {mae}')



image 1/1 /hy-tmp/yolo_soybean/dataset/outdoor/val/images/D12E-D13E_DSC04534_b_8_D12E_b_3.png: 640x256 19 1spps, 71 2spps, 11 3spps, 14.3ms
Speed: 1.5ms preprocess, 14.3ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 256)

image 1/1 /hy-tmp/yolo_soybean/dataset/outdoor/val/images/D12E-D13E_DSC04534_b_9_D12E_b_4.png: 640x224 24 1spps, 51 2spps, 10 3spps, 78.5ms
Speed: 1.3ms preprocess, 78.5ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 224)

image 1/1 /hy-tmp/yolo_soybean/dataset/outdoor/val/images/D12E-D13E_DSC04537_b_6_D12E_b_1.png: 640x288 23 1spps, 79 2spps, 10 3spps, 12.2ms
Speed: 1.7ms preprocess, 12.2ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 288)

image 1/1 /hy-tmp/yolo_soybean/dataset/outdoor/val/images/D12E-D13E_DSC04537_b_7_D12E_b_2.png: 640x192 25 1spps, 49 2spps, 3 3spps, 12.1ms
Speed: 1.4ms preprocess, 12.1ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 192)

image 1/1 /hy-tmp/yolo_soybean/dataset/outdoor/v

### Predict pods

In [ ]:
def load_labels(label_path):
    """Load label file and count the number of beans (detection boxes)."""
    with open(label_path, 'r') as f:
        lines = f.readlines()
    return len(lines)

def count_beans(predictions):
    """Count the number of beans (detection boxes) in predictions."""
    return len(predictions)

import glob
import os
import numpy as np

test_images_path = '/hy-tmp/yolo_soybean/datasets/Normal_dataset/eval/images'
labels_path = '/hy-tmp/yolo_soybean/datasets/Normal_dataset/eval/labels'

image_paths = glob.glob(os.path.join(test_images_path, '*.*'))

errors = []
results = []

for image_path in image_paths:
    image_name = os.path.basename(image_path)
    label_name = os.path.splitext(image_name)[0] + '.txt'
    label_path = os.path.join(labels_path, label_name)

    if not os.path.exists(label_path):
        print(f'Label file not found for image {image_name}, skipping...')
        continue

    true_count = load_labels(label_path)
    model_results = model(image_path)

    if model_results and len(model_results) > 0:
        predictions = model_results[0].boxes.data  
        pred_count = count_beans(predictions.cpu().numpy())  
    else:
        pred_count = 0

    error = abs(pred_count - true_count)
    errors.append(error)
    results.append((image_name, true_count, pred_count, error))

# Sort results by error in descending order
results.sort(key=lambda x: x[3], reverse=True)

# Write results to a file
with open('sorted_errors_pods.txt', 'w') as file:
    for result in results:
        file.write(f'Image: {result[0]}, True Count: {result[1]}, Predicted Count: {result[2]}, Error: {result[3]}\n')

mae = np.mean(errors)
print(f'Mean Absolute Error (MAE): {mae}')



# Save predicted labels

In [9]:
import os
import glob
from PIL import Image

# Define the paths
test_images_path = '/hy-tmp/yolo_soybean/dataset/Normal_dataset/eval/images'
predictions_path = '/hy-tmp/yolo_soybean/predictions_3'  # Directory to save predictions

# Ensure the predictions directory exists
os.makedirs(predictions_path, exist_ok=True)

# Get all image paths
image_paths = glob.glob(os.path.join(test_images_path, '*.*'))

# Iterate over each image
for image_path in image_paths:
    # Predict and save results
    results = model.predict(image_path, save_txt=True, save=True)  # Ensure 'save' is set to True


print("Predictions saved with bounding boxes drawn on the images.")



image 1/1 /hy-tmp/yolo_soybean/dataset/outdoor/val/images/soybean130.jpg: 640x256 1 1spp, 50 2spps, 57 3spps, 4 4spps, 11.6ms
Speed: 7.7ms preprocess, 11.6ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 256)
Results saved to runs/detect/predict
1 label saved to runs/detect/predict/labels

image 1/1 /hy-tmp/yolo_soybean/dataset/outdoor/val/images/soybean131.jpg: 640x192 4 1spps, 29 2spps, 28 3spps, 2 4spps, 10.6ms
Speed: 1.3ms preprocess, 10.6ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 192)
Results saved to runs/detect/predict
2 labels saved to runs/detect/predict/labels

image 1/1 /hy-tmp/yolo_soybean/dataset/outdoor/val/images/soybean135.jpg: 640x288 10 1spps, 40 2spps, 27 3spps, 1 4spp, 11.3ms
Speed: 1.8ms preprocess, 11.3ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 288)
Results saved to runs/detect/predict
3 labels saved to runs/detect/predict/labels

image 1/1 /hy-tmp/yolo_soybean/dataset/outdoor/val/images/soybean146.jpg: 640x12